<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Emilio Roa



### **Link de repositorio de GitHub:** `https://github.com/ERoa98/MDS7202-Emilio-Roa`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

  Attempting uninstall: plotly
    Found existing installation: plotly 5.10.0
    Uninstalling plotly-5.10.0:
      Successfully uninstalled plotly-5.10.0


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [79]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


In [97]:
df_players.dropna(subset = ['Club_Position'], inplace=True)

df_players.isnull().sum()

Name                     0
Nationality              0
National_Position    16512
Club_Position            0
Height                   0
Weight                   0
Preffered_Foot           0
Age                      0
Work_Rate                0
Weak_foot                0
Skill_Moves              0
Ball_Control             0
Dribbling                0
Marking                  0
Sliding_Tackle           0
Standing_Tackle          0
Aggression               0
Reactions                0
Interceptions            0
Vision                   0
Composure                0
Crossing                 0
Short_Pass               0
Long_Pass                0
Acceleration             0
Speed                    0
Stamina                  0
Strength                 0
Balance                  0
Agility                  0
Jumping                  0
Heading                  0
Shot_Power               0
Finishing                0
Long_Shots               0
Curve                    0
Freekick_Accuracy        0
P

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [98]:
### Código aquí ###
mask = df_players['National_Position'].isna()
df_players['Seleccionado'] = mask.apply(lambda x: 1 if x==False else 0)

df_players['Seleccionado'].value_counts()

0    16512
1     1075
Name: Seleccionado, dtype: int64

**Respuesta:**
Es posible observar un desbalance en las etiquetas, ya que la mayoría de los jugadores no son seleccionados nacionales. Del total de 17.588 jugadores solo 1075 son seleccionados nacionales.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [99]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

ct = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Club_Position', 'Preffered_Foot', 'Work_Rate', 'Nationality']),
        ('StandardScaler', StandardScaler(),
         ['Height', 'Weight', 'Age', 
          'Ball_Control', 'Dribbling',
          'Marking', 'Sliding_Tackle', 'Standing_Tackle',
          'Aggression', 'Reactions', 'Interceptions', 'Vision',
          'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
          'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
          'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
          'Freekick_Accuracy', 'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Weak_foot', 'Skill_Moves'])])


**Respuesta:**

Las justificaciones del preprocesamiento a utilizar sobre cada columna son: Se eliminaran las columnas de:

- Name: dado que el nombre no debería tener influencia en si es seleccionado o no.
- National_Position: dado que fue utilizado para crear el label.


Se hará un OneHotEncoding a las columnas de:

- Club_position: dado que para las distintas posiciones son distintos stats los que importan, lo cual debe tomar en consideración el modelo.
- Preffered_Foot: dado que es importante saber que pie utiliza para patiar y relacionarlo con la posición.
- Work_Rate: dado que es importante poseer el ratio de trabajo que tiene. Se podría pasar a ordinal pero esta complejo dado que considera la relación de 2 ratios.
- Nationality: dado que para ver si un jugador es seleccionado nacional es útil saber su nacionalidad.

Se hara una estandarización sobre las siguientes columnas dado a que siguen una distribución ajustable a una normal:

- Height
- Weight
- Age
- Ball_Control
- Dribbling
- Marking
- Sliding_Tackle
- Standing_Tackle
- Aggression
- Reactions
- Interceptions
- Vision
- Composure
- Crossing
- Short_Pass
- Long_Pass
- Acceleration
- Speed
- Stamina
- Strength
- Balance
- Agility
- Jumping
- Heading
- Shot_Power
- Finishing
- Long_Shots
- Curve
- Freekick_Accuracy
- Penalties
- Volleys'

Se hara una escalamiento de mínimo y máximo sobre las siguientes columnas dado que poseen sus límites bien definidos:

- Weak_foot
- Skill_Moves

In [100]:
### Código aquí ###
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

pipe1 = Pipeline(
    [
    ("preprocesamiento", ct),
    ("tree", DecisionTreeClassifier(criterion="entropy"))
    ]
)

pipe2 = Pipeline(
    [
    ("preprocesamiento", ct),
    ("SVM", SVC(gamma='auto'))]
)

pipe3 = Pipeline(
    [
    ("preprocesamiento", ct),
    ("RF", RandomForestClassifier(max_depth=10, random_state=0))]
)

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [101]:
### Código aquí ###
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    df_players.drop(["Seleccionado", "National_Position", "Name"], axis=1), df_players.Seleccionado, test_size=0.33, shuffle=True, stratify=df_players.Seleccionado
)

In [102]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11783, 37)
(11783,)
(5804, 37)
(5804,)


In [104]:
# distribución original
df_players.Seleccionado.value_counts() / df_players.Seleccionado.count() * 100


0    93.887531
1     6.112469
Name: Seleccionado, dtype: float64

In [105]:
# conjunto de entrenamiento
y_train.value_counts() / y_train.count() * 100

0    93.889502
1     6.110498
Name: Seleccionado, dtype: float64

In [106]:
# conjunto de pruebas
y_test.value_counts() / y_test.count() * 100

0    93.883529
1     6.116471
Name: Seleccionado, dtype: float64

In [107]:
pipe1 = pipe1.fit(X_train, y_train)
pipe2 = pipe2.fit(X_train, y_train)
pipe3 = pipe3.fit(X_train, y_train)

In [108]:
y_pred_1 = pipe1.predict(X_test)
y_pred_2 = pipe2.predict(X_test)
y_pred_3 = pipe3.predict(X_test)


In [109]:
from sklearn.metrics import confusion_matrix
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_1), "\n")
print(classification_report(y_test, y_pred_1))

Matriz de confusión

 [[5242  207]
 [ 247  108]] 

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      5449
           1       0.34      0.30      0.32       355

    accuracy                           0.92      5804
   macro avg       0.65      0.63      0.64      5804
weighted avg       0.92      0.92      0.92      5804



In [110]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_2), "\n")
print(classification_report(y_test, y_pred_2))

Matriz de confusión

 [[5449    0]
 [ 355    0]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5449
           1       0.00      0.00      0.00       355

    accuracy                           0.94      5804
   macro avg       0.47      0.50      0.48      5804
weighted avg       0.88      0.94      0.91      5804



d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [111]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_3), "\n")
print(classification_report(y_test, y_pred_3))

Matriz de confusión

 [[5443    6]
 [ 341   14]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5449
           1       0.70      0.04      0.07       355

    accuracy                           0.94      5804
   macro avg       0.82      0.52      0.52      5804
weighted avg       0.93      0.94      0.91      5804



**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

Comente de resultados observados y si es posible mejorar los resultados.
En cuanto a los resultados, en general los modelos ensamblados (RF y Desition Tree) fueron los que mejor se comportaron al predecir los seleccionados nacionales (a pesar del balance de clases), destacando por sobretodo su precisión sobre ellos. A continuación, respondiendo algunas preguntas:

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?

El valor de accuracy indica que porcentaje, de todas las clasificaciones realizadas en el conjunto de testeo, fue hechas correctamente.

La precisión, para el caso de la clase positiva, indica que porcentaje de todas las clasificaciones marcadas positivas son realmente positivas (es decir cuantos de los clasificados como seleccionados nacionales en verdad lo son).

El recall, para el caso de la clase positiva, indica que porcentaje de todas las muestras etiquetadas positivas fueron clasificadas cómo positivas (es decir cuantos de los seleccionados nacionales fueron clasificados como seleccionados nacionales).

- ¿Podrían mejorarse los resultados?, ¿Cómo?

Sí, para mejorar estos resultados se pueden ver que características aportan menos información al modelo y eliminarlas para que este se centre en lo que realmente importa, modificar los hiperparámetros del modelo a tráves de un conjunto de validación, probar otros modelos y hacer validación cruzada para ver cuál es el que mejor se adapta a los datos, y también otro punto que se discutirá en la siguiente pregunta.

- ¿Influye la cantidad de ejemplos por clase?

Sí influye, cómo se puede ver en los resultados de los 3 pipelines, la clase mayoritaria (correspondiente a no ser un seleccionado), fue la que obtuvo mejores métricas tanto en precision como en recall, lo que nos dice que todos los modelos, dado que esta clase poseía muchos más datos que la otra, se sobre ajustaron para predecir a los no seleccionados. Esto también es un punto de mejora de los resultados, dado que mediante técnicas como hacer OverSampling, UnderSamplig o darle pesos a las clases al momento de entrenar el modelo se puede corregir este sobreajuste a la clase mayoritaria.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [112]:
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

def map_position(x):
    if x in ataque:
        return 'ataque'
    elif x in central_ataque:
        return 'central_ataque'
    elif x in central:
        return 'central'
    elif x in central_defensa:
        return 'central_defensa'
    elif x in defensa:
        return 'defensa'
    elif x in arquero:
        return 'arquero'

### Drop Sub y Res
df_players_filter = df_players[~df_players["Club_Position"].isin(['Sub','Res'])].copy()

### Mapeo posiciones
df_players_filter["Club_Position"] = df_players_filter["Club_Position"].apply(map_position)

### Eliminando NAN
df_players_filter=df_players_filter[~df_players_filter.Club_Position.isna()]

df_players_filter

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado
0,Cristiano Ronaldo,Portugal,LS,central_ataque,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,central_ataque,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,central_ataque,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,LS,ataque,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,arquero,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17298,Raymond Skjelde,Norway,NaN,defensa,185,75,Right,18,Medium / Medium,2,...,65,44,35,22,25,24,22,41,31,0
17301,Andrew O'Connor,Republic of Ireland,NaN,defensa,175,73,Left,21,Medium / Medium,3,...,55,43,26,28,29,30,30,31,20,0
17310,Tom Bolarinwa,England,NaN,central,180,80,Right,26,Medium / Medium,3,...,54,22,38,37,37,37,32,29,29,0
17312,Zak Mills,England,NaN,defensa,182,83,Right,24,Medium / Medium,4,...,61,43,22,24,28,30,32,39,31,0


In [113]:
df_players_filter.Club_Position.value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

Es posible notar un claro desbalance entre las clases, siendo las clases mayoritarias la de defensa y central, mientras que la clase minoritaria es la de central_defensa. 

In [114]:
ct2 = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Preffered_Foot', 'Work_Rate', 'Nationality']),
        ('StandardScaler', StandardScaler(),
         ['Height', 'Weight', 'Age', 
          'Ball_Control', 'Dribbling',
          'Marking', 'Sliding_Tackle', 'Standing_Tackle',
          'Aggression', 'Reactions', 'Interceptions', 'Vision',
          'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
          'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
          'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
          'Freekick_Accuracy', 'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Weak_foot', 'Skill_Moves'])])

In [117]:
pipe1 = Pipeline(
    [
    ("preprocesamiento", ct2),
    ("tree", DecisionTreeClassifier(criterion="entropy"))
    ]
)

pipe2 = Pipeline(
    [
    ("preprocesamiento", ct2),
    ("SVM", SVC(gamma='auto'))]
)

pipe3 = Pipeline(
    [
    ("preprocesamiento", ct2),
    ("RF", RandomForestClassifier(max_depth=10, random_state=0))]
)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(
    df_players_filter.drop(["Club_Position", "National_Position", "Name"], axis=1), df_players_filter.Club_Position, test_size=0.33, shuffle=True, stratify=df_players_filter.Club_Position
)

In [125]:
pipe1 = pipe1.fit(X_train, y_train)
pipe2 = pipe2.fit(X_train, y_train)
pipe3 = pipe3.fit(X_train, y_train)

In [126]:
y_pred_1 = pipe1.predict(X_test)
y_pred_2 = pipe2.predict(X_test)
y_pred_3 = pipe3.predict(X_test)

In [131]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_1), "\n")
print(classification_report(y_test, y_pred_1))


Matriz de confusión

 [[209   0   0   0   0   0]
 [  0  89  21  32   0   0]
 [  0  19 137  97  15  31]
 [  0  16  89  70   7  10]
 [  0   0  17   1  11  40]
 [  0   2  33  10  30 314]] 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.71      0.63      0.66       142
        central       0.46      0.46      0.46       299
 central_ataque       0.33      0.36      0.35       192
central_defensa       0.17      0.16      0.17        69
        defensa       0.79      0.81      0.80       389

       accuracy                           0.64      1300
      macro avg       0.58      0.57      0.57      1300
   weighted avg       0.64      0.64      0.64      1300



In [129]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_2), "\n")
print(classification_report(y_test, y_pred_2))


Matriz de confusión

 [[209   0   0   0   0   0]
 [  0 115  23   4   0   0]
 [  0  18 222   8   0  51]
 [  0  10 151  19   0  12]
 [  0   0  10   0   0  59]
 [  0   0  11   0   0 378]] 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.80      0.81      0.81       142
        central       0.53      0.74      0.62       299
 central_ataque       0.61      0.10      0.17       192
central_defensa       0.00      0.00      0.00        69
        defensa       0.76      0.97      0.85       389

       accuracy                           0.73      1300
      macro avg       0.62      0.60      0.57      1300
   weighted avg       0.69      0.73      0.67      1300



In [132]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_3), "\n")
print(classification_report(y_test, y_pred_3))

Matriz de confusión

 [[209   0   0   0   0   0]
 [  0 109  30   3   0   0]
 [  0  15 211  21   1  51]
 [  0  10 126  40   1  15]
 [  0   0   4   0   4  61]
 [  0   0   8   1   0 380]] 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.81      0.77      0.79       142
        central       0.56      0.71      0.62       299
 central_ataque       0.62      0.21      0.31       192
central_defensa       0.67      0.06      0.11        69
        defensa       0.75      0.98      0.85       389

       accuracy                           0.73      1300
      macro avg       0.73      0.62      0.61      1300
   weighted avg       0.73      0.73      0.70      1300



**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

Sobre los resultados de esta clasificación, en cuanto a precision se puede ver que para todos los modelos implementados la clase más fácil de clasificar es la de arquero, esto se puede deber a que resulta ser la posición más apartada del resto en cuanto a características de juego lo cual podría tener una inferencia directa por sobre las características físicas de los jugadores. Se debe tener ojo también en un posible sobreajuste dentro de esta variable al obtener métricas prácticamente perfectas.

Otro comentario relevante puede ser el hecho de que las clases se encuentran sumamente desbalanceadas, por ejemplo la diferencia entre la clase mayoritaria y la clase minoritaria es de 1180 vs 209 respectivamente influyendo de manera directa en el aprendizaje del modelo, lo cual se refleja en las métricas de precision, recall y f1-score. En este sentido, al ver tan buenos resultados en las métricas sobre la clase mayoritaria también se podría mencionar la presencia de un sobre entrenamiento del modelo sobre esta posición.

Ahora, sobre la pregunta de si es posible mejorar los resultados, se puede volver a destacar el hecho del desbalance de clases por lo que se cree que aplicando técnicas de tratamiento para este problema o modelos de entrenamiento mejor preparados para estos casos, la predicción podría presentar mejoras sustanciales.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica $R^2$ es una medida estadística que muestra qué tan cerca se encuentran los puntos de datos sobre la linea de regresión. Esto hace referencia principalmente al porcentaje de variación en la variable de respuesta que es explicado por un modelo lineal.

Los valores que ofrece esta métrica se encuentran en los valores 0 y 1 en donde el 0 indica que el modelo no es capaz de explicar ninguna porción de variabilidad de los datos, mientras que el valor 1 indica que el modelo explica absolutamente toda la variabilidad de los datos en torno a su media.

In [150]:

df_sueldos = pd.read_csv('sueldos.csv')
df_sueldos.rename(columns={"Player":'Name'},inplace=True)
df_sueldos.drop(['Unnamed: 0'], axis=1, inplace=True)

df_sueldos

,Name,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0
3,Neymar,797726.0
4,Gareth Bale,607706.0
...,...,...
2529,Simone Ghidotti,283.0
2530,Valentin Gendrey,100.0
2531,John Yeboah,70.0
2532,Antonis Aidonis,70.0


In [152]:
df_final = df_sueldos.merge(df_players, on="Name" , how="inner")
df_final

,Name,Weekly Salary,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado
0,Lionel Messi,1538905.0,Argentina,RW,RW,170,72,Left,29,Medium / Medium,...,68,71,85,95,88,89,90,74,85,1
1,Cristiano Ronaldo,1248536.0,Portugal,LS,LW,185,80,Right,32,High / Low,...,95,85,92,93,90,81,76,85,88,1
2,Antoine Griezmann,997032.0,France,CAM,RS,176,67,Left,26,High / Medium,...,87,80,81,88,82,84,75,71,87,1
3,Neymar,797726.0,Brazil,LW,LW,174,68,Right,25,High / Medium,...,61,62,78,89,77,79,84,81,83,1
4,Gareth Bale,607706.0,Wales,RS,RW,183,74,Left,27,High / Medium,...,85,86,91,87,90,86,85,76,76,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Michael Langer,903.0,Austria,NaN,GK,194,92,Right,32,Medium / Medium,...,50,14,15,14,13,12,12,13,11,0
1862,Nicolas Lemaître,859.0,France,NaN,Res,185,83,Right,20,Medium / Medium,...,32,23,23,17,15,19,14,22,17,0
1863,Andrea Pinamonti,805.0,Italy,NaN,Res,181,72,Right,17,Medium / Low,...,65,62,57,74,60,43,43,69,43,0
1864,Tim Wiesner,674.0,Germany,NaN,Res,192,87,Right,20,Medium / Medium,...,57,16,19,13,15,18,16,20,14,0


In [163]:
### Código aquí ###
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(
    df_final.drop(["National_Position", "Name"], axis=1), df_final['Weekly Salary'], test_size=0.33, shuffle=True
)

ct3 = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Club_Position','Preffered_Foot', 'Work_Rate', 'Nationality']),
        ('StandardScaler', StandardScaler(),
         ['Height', 'Weight', 'Age', 
          'Ball_Control', 'Dribbling',
          'Marking', 'Sliding_Tackle', 'Standing_Tackle',
          'Aggression', 'Reactions', 'Interceptions', 'Vision',
          'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
          'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
          'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
          'Freekick_Accuracy', 'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Weak_foot', 'Skill_Moves'])])


pipe4 = Pipeline(
    [
    ("preprocesamiento", ct3),
    ("tree",  RandomForestRegressor(max_depth=5, random_state=0))
    ]
)

pipe5 = Pipeline(
    [
    ("preprocesamiento", ct3),
    ("tree",  LinearRegression())
    ]
)


In [164]:
pipe4 = pipe4.fit(X_train, y_train)
pipe5 = pipe5.fit(X_train, y_train)

In [165]:
y_pred4 = pipe4.predict(X_test)
y_pred5 = pipe5.predict(X_test)


In [166]:
from sklearn.metrics import r2_score

def evaluate(y_test, y_pred4):
    print('R²:', r2_score(y_test, y_pred4))
    
evaluate(y_test, y_pred4)

R²: 0.5737908584513322


In [167]:
evaluate(y_test, y_pred5)

R²: 0.129323170729443


**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

Dados los resultados que ofrece la métrica R^2, se tiene que es claramente posible mejorar los resultados obtenidos, un R^2 = 0.57 se encuentra aún bastante alejado de 1. La cantidad de datos que se tienen para entrenar el modelo puede resultar clave para la mejora. Dada la gran falta datos sobre el salario de muchos jugadores o la falta de información en jugadores de los cuales se tiene su salario, el dataset trabajado contaba solamente con cerca de 1800 datos, lo cual es bastante poco para poder tener una caracterización concreta sobre la predicción de salarios en los jugadores. Es por esto que resultaría de bastante ayuda en la mejora de los resultados un mayor volumen de datos.

Ahora, en cuanto a las características también podría haber un par más de ellas que permitan entender de mejor manera el comportamiento de los salarios. Por dar algunos ejemplos, existen algunas características que podrían tener una mayor correlación con los salarios como el club en el que juega (dado que el club es el que le paga al jugador), la liga en la que se desempeña (diferenciando ligas de primera, segunda y las diferentes divisiones profesionales) y por último algunas características que se tienen dentro del año/temporada del desempeño del jugador como los goles, asistencias, salvadas, balones recuperados, regates... entre muchas otras estadísticas.



# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>